## Generating comments on the poems with VLLM

VLLM allows for batched processing and much higher throughput from LLMs. 

https://vllm.readthedocs.io/en/latest/

They have a very active discord server and folks are very helpful. I would strongly recommend hanging out there because the documentation doesn't cover some common snags and most of these have been spoken about on the server. 

In [ ]:
%pip install vllm pandas ipywidgets 


In [1]:
import pandas as pd


In [27]:
df = pd.read_csv("data/all-poems-for-vllm.csv")


Took a lot of iterations with various models to finally settle on wizard vicuna and this particular prompt. the amount of difference even small changes in the instructions can make is mind-boggling. 

In [38]:
prefix = "A chat between a curious human and an artificial intelligence assistant. The assistant gives thoughtful and smart answers to the human's questions.\n\n"
instruction = " ### Human: You are an expert poetry critic. What are the emotions, themes and tone of this poem?  Provide the answer as a list"
suffix = " ### Assisant: "


In [39]:
def format_poems(poems):
    return f"{prefix}{poems}. {instruction}{suffix}"


In [40]:
df["formatted_text"] = df["cleaned_content"].apply(format_poems)

In [41]:
df.rename(columns={"url_num": "poem_id"}, inplace=True)


In [42]:
df.head()

,cleaned_content,poem_id,truncated_text,formatted_text
0,"Well, friend, we’re here again — sauntering th...",58566,"Well, friend, we’re here again — sauntering th...",A chat between a curious human and an artifici...
1,Grieve not that winter masks the yet quick ear...,53997,Grieve not that winter masks the yet quick ear...,A chat between a curious human and an artifici...
2,who can bear the idea of Eternal Recurrence? I...,27139,who can bear the idea of Eternal Recurrence? I...,A chat between a curious human and an artifici...
3,"Prosaic miles of streets stretch all round, As...",53778,"Prosaic miles of streets stretch all round, As...",A chat between a curious human and an artifici...
4,Across the deep eternal sky a thousand changin...,152076,Across the deep eternal sky a thousand changin...,A chat between a curious human and an artifici...


In [43]:
df.drop(columns=["cleaned_content"], inplace=True)


In [ ]:
import numpy as np

# Although VLLM documentation says you don't need to batch the input manually,
# I found that if i didn't I would run into CUDA out of memory or system out of RAM
# errors. Took some fiddling around to figure out what size works. 
# Trained this on a RTX A5000 24GB VRAM instance
# Runtime for the batched prompts was about 2 hours.

batch_size = 64

poem_batches = [group for _, group in df.groupby(
    np.arange(len(df)) // batch_size)]


In [ ]:
formatted_prompts = [batch["formatted_text"].tolist() for batch in poem_batches]

In [1]:
from vllm import LLM, SamplingParams

MODEL = "TheBloke/wizard-vicuna-13B-AWQ"

In [ ]:
llm = LLM(model=MODEL, trust_remote_code=True, quantization="awq")

In [188]:
sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    best_of=5,
    top_k=50,
    max_tokens=150,
)

#took a lot of fiddling with the parameters to ensure that there wasn't much repetition and that the answers were coherent. 


In [212]:
# There were many many of these trial runs to ensure that the prompts were good and the results were consistant
trial_outputs = llm.generate(df.iloc[8974].formatted_text, sampling_params)

Processed prompts: 100%|██████████| 1/1 [01:28<00:00, 88.57s/it]


In [215]:
for output in trial_outputs:
    print(output.trial_outputs[0].text)

1. Emotions: Amusement, irony, wit, and humor.
2. Themes: Honesty, morality, policy, and the nature of language.
3. Tone: Playful, sarcastic, and philosophical.


In [ ]:
"""
Generate and assign text outputs to a DataFrame based on formatted prompts.

This code processes formatted prompts in batches, generates text outputs, and assigns them to a DataFrame for poems with matching IDs.
It iterates through each batch of prompts and associates the generated text with the corresponding poem in the DataFrame.

Parameters:
- formatted_prompts (list): A list of formatted prompts to generate text from.
- llm: The language model used for text generation.
- sampling_params: Parameters for controlling the sampling strategy during text generation.
- poem_batches (list): A list of batches containing poem data, each with a "poem_id" attribute.
- df (DataFrame): The DataFrame containing poem data and a "generated_text" column to store the generated output.

"""
for i, batch in enumerate(formatted_prompts):
    # Generate the output
    outputs = llm.generate(batch, sampling_params)
    # For each poem in the batch
    for j, poem_id in enumerate(poem_batches[i]["poem_id"]):
        # Assign the generated text to the "generated_text" column in the dataframe
        df.loc[df["poem_id"] == poem_id, "generated_text"] = outputs[j].outputs[0].text